# Automatic Sandpit Refinement for D-Flow FM

This notebook provides an automated workflow for refining unstructured grids around sandpit areas in D-Flow FM models. The refinement process uses Casulli refinement to gradually transition from coarse background resolution to fine target resolution.

## Workflow Overview

1. **Configuration**: Set file paths and refinement parameters
2. **Load Grid and Polygons**: Load the grid and create/load sandpit polygons
3. **Plan Refinement**: Analyze grid resolution and generate refinement zones
4. **Execute Refinement**: Apply Casulli refinement to the grid
5. **Monitor Quality** (Optional): Analyze grid quality metrics

## Requirements

- `dfm_tools`
- `meshkernel`
- `numpy`
- `matplotlib`
- `shapely`

In [1]:
# Step 0: Configuration and Setup

import numpy as np
import os
import dfm_tools as dfmt
import matplotlib.pyplot as plt

# Import utility functions
from src.polygon_utils import InteractivePolygonDrawer, load_pol_file, save_pol_file, generate_refinement_polygons, expand_polygon_outward
from src.refinement_utils import compute_refinement_steps, apply_casulli_refinement, print_refinement_summary
from src.visualization_utils import plot_grid, set_interactive_plots, set_static_plots
from src.monitoring_utils import analyze_grid_quality, plot_grid_quality

# Base path is the workspace root in Codespace
main_path = '/workspaces/orelse-sandpit-automation'  # or just use '.'
nc_file = 'dcsm_0_125nm_2ref_bathygr7_RGFGRID_net.nc'
nc_path = os.path.join(main_path, 'data', 'input', nc_file)

# Output settings
output_dir = os.path.join(main_path, 'data', 'output')
os.makedirs(output_dir, exist_ok=True)

# User configuration
use_existing_pol_file = True
existing_pol_file = os.path.join(main_path, 'data', 'input', 'sandpits.pol')

# Refinement parameters
target_resolution = 30  # Target resolution in meters for the finest refinement level
buffer_around_sandpit = 250  # Buffer around sandpit polygons in meters
N = 7  # Number of transition cells

print("Configuration loaded")
print(f"Target resolution: {target_resolution} m")
print(f"Buffer distance: {buffer_around_sandpit} m")
print(f"Transition cells: {N}")

Configuration loaded
Target resolution: 30 m
Buffer distance: 250 m
Transition cells: 7


## Step 1: Load Grid and Define Sandpit Polygons

This step loads the D-Flow FM grid and either:
- Loads existing sandpit polygons from a .pol file
- Opens an interactive drawing tool to create new polygons

The polygons define the areas where refinement will be applied.

In [2]:
# Step 1: Load Grid and Create/Load Polygons

# Load grid data
print("Loading grid...")
ugrid = dfmt.open_partitioned_dataset(nc_path)
ugrid_original = dfmt.open_partitioned_dataset(nc_path)  # Keep original for later comparison

# Convert to meshkernel objects for consistent plotting and refinement
mk_object = ugrid.grid.meshkernel
mk_backup = ugrid_original.grid.meshkernel  # Keep backup

print("Loaded grid using dfm_tools and converted to meshkernel")
print(f"Grid dimensions: {ugrid.sizes}")

# Handle polygon input/creation
if use_existing_pol_file:
    # Load existing polygon file
    pol_path = os.path.join(output_dir, existing_pol_file)
    if os.path.exists(pol_path):
        polygons = load_pol_file(pol_path)
        print(f"Loaded {len(polygons)} polygons from {existing_pol_file}")
    else:
        print(f"File {existing_pol_file} not found, switching to interactive mode")
        use_existing_pol_file = False

if not use_existing_pol_file:
    # Interactive polygon creation
    set_interactive_plots()
    print("Opening interactive plot...")
    print("Instructions:")
    print("- RIGHT CLICK: add polygon vertex")
    print("- ENTER: finish current polygon")
    print("- Close window when done")
    
    drawer = InteractivePolygonDrawer(ugrid, nc_path)
    polygons = drawer.draw_polygons()
    
    if polygons:
        output_file = os.path.join(output_dir, 'sandpits.pol')
        save_pol_file(polygons, output_file)
        print(f"Saved {len(polygons)} polygons to {output_file}")

Loading grid...
>> xu.open_dataset() with 1 partition(s): 1 [nodomainvar] : 0.22 sec
>> xu.open_dataset() with 1 partition(s): 1 [nodomainvar] : 0.16 sec
Loaded grid using dfm_tools and converted to meshkernel
Grid dimensions: Frozen({'nmesh2d_node': 843868, 'nmesh2d_edge': 1679800, 'Two': 2, 'nmesh2d_face': 835646, 'max_nmesh2d_face_nodes': 4})
Loaded 4 polygons from /workspaces/orelse-sandpit-automation/data/input/sandpits.pol


## Step 2: Plan Refinement Strategy

This step:
1. Analyzes the current grid resolution within the sandpit polygons
2. Calculates the number of refinement steps needed
3. Generates refinement zones with automatic overlap detection and merging
4. Visualizes the refinement plan

The refinement zones are created from coarse (outer) to fine (inner) resolution.

In [3]:
# Step 2: Plan Refinement Strategy

# Analyze grid resolution and compute refinement steps
print("\nAnalyzing grid resolution within sand pit polygons...")
refinement_params = compute_refinement_steps(ugrid, target_resolution, polygons)

print("\nRefinement planning:")
print(f"Current grid spacing: {refinement_params['current_spacing_m']:.1f} m")
print(f"Target resolution: {refinement_params['target_resolution']} m") 
print(f"Number of refinement steps needed: {refinement_params['n_steps']}")

print("\nEnvelope sizes (meters):")
for i, size in enumerate(refinement_params['envelope_sizes_m']):
    print(f"  Step {i}: {size:.1f} m")

# Generate refinement polygons with overlap merging
(all_refinement_polygons, all_original_polygons, 
 buffer_polygons, expansions) = generate_refinement_polygons(
    polygons, refinement_params, buffer_around_sandpit, N)

# Store original buffer polygons for visualization
original_buffer_polygons = []
for i, polygon in enumerate(polygons):
    polygon_array = np.array(polygon)
    center_lat = np.mean(polygon_array[:, 1])
    expanded_polygon = expand_polygon_outward(polygon, buffer_around_sandpit, center_lat)
    original_buffer_polygons.append(expanded_polygon)

# Visualize refinement plan
set_static_plots()
plot_grid(mk_object, polygons, all_refinement_polygons, all_original_polygons,
          buffer_polygons, refinement_params['envelope_sizes_m'], refinement_params['n_steps'],
          original_buffer_polygons=original_buffer_polygons,
          title='D-Flow FM Grid with Sand Pit Polygons and Merged Refinement Zones')
plt.show()


Analyzing grid resolution within sand pit polygons...

Refinement planning:
Current grid spacing: 460.0 m
Target resolution: 30 m
Number of refinement steps needed: 4

Envelope sizes (meters):
  Step 0: 460.0 m
  Step 1: 230.0 m
  Step 2: 115.0 m
  Step 3: 57.5 m
  Step 4: 28.8 m
Generating refinement polygons with overlap merging:
Current spacing: 460 m
Target resolution: 30 m
Number of refinement steps: 4
Buffer around sandpit: 250 m
Step 1 - refine to 28.8 m, transition width: 258.8 m, total expansion: 508.8 m
Step 2 - refine to 57.5 m, transition width: 517.5 m, total expansion: 1026.2 m
Step 3 - refine to 115.0 m, transition width: 1035.0 m, total expansion: 2061.2 m
Step 4 - refine to 230.0 m, transition width: 2070.0 m, total expansion: 4131.2 m

Expansions from sandpit (outermost to innermost): [4131.2, 2061.2, 1026.2, 508.8]
Step 1: Checking 4 polygons for overlaps...
  Merged 2 overlapping polygons into 1 polygon
  Step 1: 4 polygons → 3 polygons
  Resolution: 28.8m, expansi

## Step 3: Execute Grid Refinement

This step applies Casulli refinement to the meshkernel object using the generated refinement zones. The refinement is applied from outside to inside (coarse to fine) to ensure smooth transitions.

After refinement, the refined grid is visualized showing the final mesh structure.

In [ ]:
# Step 3: Execute Grid Refinement

# Apply Casulli refinement
print("\nApplying Casulli refinement...")
apply_casulli_refinement(mk_object, all_refinement_polygons)

# Visualize refined grid
set_static_plots()
plot_grid(mk_object, polygons, all_refinement_polygons, all_original_polygons,
          buffer_polygons, refinement_params['envelope_sizes_m'], refinement_params['n_steps'],
          title='Refined Grid after Casulli Refinement with Merged Overlapping Zones')

# Print refinement summary
print_refinement_summary(polygons, all_refinement_polygons, 
                        refinement_params['envelope_sizes_m'], 
                        refinement_params['n_steps'], buffer_polygons)

# Save refined grid if needed
# ugrid_refined = dfmt.meshkernel_to_UgridDataset(mk_object, crs='EPSG:4326')
# output_nc = os.path.join(output_dir, 'refined_grid.nc')
# ugrid_refined.to_netcdf(output_nc)
# print(f"\nRefined grid saved to: {output_nc}")

## Step 4: Monitor Grid Quality (Optional)

This optional step analyzes the quality of the refined grid by examining:
- **Resolution**: Face areas converted to characteristic lengths
- **Smoothness**: Ratio of adjacent cell sizes (target < 1.4)
- **Orthogonality**: Deviation from 90° angles (target < 0.01)

Note: This analysis can be computationally intensive for large grids.

In [ ]:
# Step 4: (Optional) Monitor Grid Quality

# Uncomment the following lines to perform grid quality analysis
# Note: This can be computationally intensive for large grids

print("\nAnalyzing grid quality...")
quality_data = analyze_grid_quality(mk_object, ugrid_original, all_refinement_polygons, polygons)

set_static_plots()
plot_grid_quality(quality_data, all_refinement_polygons, target_resolution)